In [4]:
from lxml import etree
from py2neo import Graph, Node, Relationship
import csv
from os import walk
    
    
def parse_text(node, file_string, tx_list, active_node, csv_writer, date):
        text_string = node.text.replace('\t','').replace('\n','')
        text_id = f"{file_string}_{str(node.sourceline)}_{str(len(text_string))}"
        text_node = Node("text", file=file_string, id=text_id)
        tx_list.append(text_node)
        tx_list.append(Relationship(text_node, "TEXT_OF", active_node, order=node.sourceline))
        csv_writer.writerow([text_id, text_string, date])

def parse_body(current_node, current_tree, file_string, csv_writer, tx_list, date):
    for node in current_tree:
        if node.tag == 'enum':
            if current_node['enum'] is not None:
                print(f'enum > 1 @ {str(node.sourceline)}')
            current_node['enum'] = node.text
            continue
        if node.tag in ('table','metadata','form','toc','toc-entry'):
            continue
        if node.tag == 'text' and node.text is not None and len(node.text) > 100:
            parse_text(node, file_string, tx_list, current_node, csv_writer, date)
        elif node.tag == 'text' or len(node.attrib) == 0:
            parse_body(current_node, node, file_string, csv_writer, tx_list, date)
        else:
            new_node = Node(node.tag, **node.attrib)
            tx_list.append(new_node)
            new_node['file'] = file_string
            if node.text is not None and len(node.text) > 100:
                parse_text(node, file_string, tx_list, new_node, csv_writer, date)
            tx_list.append(Relationship(new_node, "CHILD_OF", current_node, order=node.sourceline))
            parse_body(new_node, node, file_string, csv_writer, tx_list, date)

def parse_root(name_end, file, ignore_files, graph):
    file = file.strip()
    start = file.index('data')
    end = file.index(name_end)
    file_part = file[start:end].replace('/', '__')
    if file_part not in ignore_files:
        tx = []
        tree = etree.parse(file)
        root = tree.getroot()
        try:
            date = tree.xpath('//action[1]/action-date[1]/@date')[0]
        except IndexError:
            return
        date = date[:4]+'-'+date[4:6]+'-'+date[6:]
        root_node = Node(root.tag, **root.attrib)
        root_node['file'] = file_part
        root_node['action_date'] = date
        tx.append(root_node)
        with open(f'/home/harry/PycharmProjects/congress/docs/{file_part}', 'w') as csv_file:
            parse_body(root_node, root, file_part, csv.writer(csv_file, delimiter='\t'), tx, date)
        transaction = graph.begin()
        for trans in tx:
            transaction.create(trans)
        transaction.commit()

In [2]:
parse_root('/text-versions'
           ,'/home/harry/PycharmProjects/congress/data/115/bills/s/s1/text-versions/pcs/BILLS-115s1pcs.xml'
           , []
           , Graph(user="neo4j", password="W8ih9^ql#38^mD$r2"))

In [5]:
_, _, filenames = next(walk('/home/harry/PycharmProjects/congress/docs/'))
g = Graph(user="neo4j", password="W8ih9^ql#38^mD$r2")

with open('/home/harry/PycharmProjects/congress/data/115/xml_todo.txt') as xml_todos:
    for file_name in xml_todos:
        parse_root('/text-versions', file_name, filenames, g)

In [23]:
_, _, filenames = next(walk('/home/harry/PycharmProjects/congress/docs/'))
g = Graph(user="neo4j", password="W8ih9^ql#38^mD$r2")

with open('/home/harry/PycharmProjects/congress/data/usc/xml_todo.txt') as xml_todos:
    count = 0
    for file_name in xml_todos:
        count += 1
        if count > 15:
            break
        parse_root('.xml', file_name, filenames, g)

In [2]:
graph = Graph(user="neo4j", password="W8ih9^ql#38^mD$r2")
transaction = graph.begin()


